In [1]:
import requests
import urllib
import pandas as pd
import pymysql
import numpy as np
import datetime

In [4]:
db_con = pymysql.connect(
    user='', 
    passwd='', 
    host='', 
    db='', 
    charset=''
)

cursor = db_con.cursor(pymysql.cursors.DictCursor)

sql = """

"""

cursor.execute(sql)
result = cursor.fetchall()
m_dt1 = pd.DataFrame(result)
cursor.close()

m_dt1

,kbm_apartment_id,number_of_units,x,y
0,1,548,127.036753,37.519380
1,2,644,127.022874,37.514887
2,3,60,127.038175,37.510650
3,4,111,127.038185,37.519716
4,5,70,127.032822,37.520645
...,...,...,...,...
57629,736315,25,127.216402,37.233771
57630,736316,17,127.216413,37.233773
57631,736318,43,129.021447,35.152604
57632,736423,19,128.087668,34.937846


In [5]:
m_dt1.isnull().sum()

kbm_apartment_id    0
number_of_units     0
x                   0
y                   0
dtype: int64

In [6]:
m_dt1 = m_dt1.dropna()
m_dt1.isnull().sum()

kbm_apartment_id    0
number_of_units     0
x                   0
y                   0
dtype: int64

In [7]:
def get_count_from_KAKAO(category_group_code, lat, long, radius):
#     start = time.time()
    MYAPP_KEY = ''
    
    headers = {
        'Content-Type': 'application/json; charset=UTF-8',
        'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)
    }
    
    category_group_code = category_group_code.encode("utf-8")

    p = urllib.parse.urlencode(
        {
            'category_group_code' : category_group_code,
            'x' : lat,
            'y' : long,
            'radius' : radius
        }
    )
    result = requests.get("https://dapi.kakao.com/v2/local/search/category.json", headers=headers, params=p)
    
    ## 일일 제공량 쿼터가 있으므로 경고문 추가
    try :
        result_f = list(result.json().values())[-1]['total_count']
        
#         global result_f
#         print("소요 시간 : ", time.time() - start)
             
    except :
        if 'limit' in result.json()['message']:
            print('KAKAO API: 일일 제공량 초과\n')
            print(result.json())
            
        else :
            print(result)
        
    return result_f

#  Error 메시지 : string indices must be integers의 경우
# 공식 에러 메시지 : {'errorType': 'RequestThrottled', 'message': 'API limit has been exceeded.'}
# => 일일 제공량 쿼터제한을 초과한 경우
# 쿼터제한 수 : 로컬 - 카테고리 장소 검색 => 일간 10만건 (월간 300만건)

## 기준정보 출처
# 출처 : https://developers.kakao.com/terms/latest/ko/site-policies#%EC%BF%BC%ED%84%B0%EB%B0%8F%EC%A0%9C%ED%95%9C
# 지도 category_group_code 확인용 출처 : https://developers.kakao.com/docs/latest/ko/local/dev-guide

In [8]:
def get_count_query_from_KAKAO(query, lat, long, radius):
#     start = time.time()
    MYAPP_KEY = ''
       
    headers = {
        'Content-Type': 'application/json; charset=UTF-8',
        'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)
    }
    
    query = query.encode("utf-8")

    p = urllib.parse.urlencode(
        {
            'query' : query,
            'x' : lat,
            'y' : long,
            'radius' : radius
        }
    )
    
 
    result = requests.get("https://dapi.kakao.com/v2/local/search/keyword.json", headers=headers, params=p)
    
    ## 일일 제공량 쿼터가 있으므로 경고문 추가
    try :
        result_f = list(result.json().values())[-1]['total_count']
#         print("소요 시간 : ", time.time() - start)
             
    except :
        if 'limit' in result.json()['message']:
            print('KAKAO API: 일일 제공량 초과\n')
            print(result.json())
        
    return result_f

#  Error 메시지 : string indices must be integers의 경우
# 공식 에러 메시지 : {'errorType': 'RequestThrottled', 'message': 'API limit has been exceeded.'}
# => 일일 제공량 쿼터제한을 초과한 경우
# 쿼터제한 수 : 로컬 - 카테고리 장소 검색 => 일간 10만건 (월간 300만건)

## 기준정보 출처
# 출처 : https://developers.kakao.com/terms/latest/ko/site-policies#%EC%BF%BC%ED%84%B0%EB%B0%8F%EC%A0%9C%ED%95%9C
# 지도 category_group_code 확인용 출처 :https://developers.kakao.com/docs/latest/ko/local/dev-guide#search-by-category-request-category-group-code

In [11]:
# 세대수 변수처리
number_of_units = 50

# 세대수별 차등 간격 정의
diff_a = 10

def cal_distance_unit(x, y):
    units = x
    dist = y
    diff = diff_a
    
    if (units >=  number_of_units) & (units < number_of_units):
        res = dist + diff
    elif (units >= number_of_units * 2) & (units < number_of_units * 3):
        res = dist + (diff*2)
    elif (units >= number_of_units * 3) & (units < number_of_units * 4):
        res = dist + (diff*3)
    elif (units >= number_of_units * 4) & (units < number_of_units * 5):
        res = dist + (diff*4)
    elif (units >= number_of_units * 5) & (units < number_of_units * 6):
        res = dist + (diff*5)
    elif (units >= number_of_units * 6) & (units < number_of_units * 7):
        res = dist + (diff*6)
    elif (units >= number_of_units * 7) & (units < number_of_units * 8):
        res = dist + (diff*7)
    elif (units >= number_of_units * 8) & (units < number_of_units * 9):
        res = dist + (diff*8)
    elif (units >= number_of_units * 9) & (units < number_of_units * 10):
        res = dist + (diff*9)
    elif (units >= number_of_units * 10) & (units < number_of_units * 11):
        res = dist + (diff*10)
    elif (units >= number_of_units * 11) & (units < number_of_units * 12):
        res = dist + (diff*11)
    elif (units >= number_of_units * 12) & (units < number_of_units * 13):
        res = dist + (diff*12)
    elif (units >= number_of_units * 13) & (units < number_of_units * 14):
        res = dist + (diff*13)
    elif (units >= number_of_units * 14) & (units < number_of_units * 15):
        res = dist + (diff*14)
    elif (units >= number_of_units * 15) & (units < number_of_units * 16):
        res = dist + (diff*15)
    elif (units >= number_of_units * 16) & (units < number_of_units * 17):
        res = dist + (diff*16)
    elif (units >= number_of_units * 17) & (units < number_of_units * 18):
        res = dist + (diff*17)
    elif (units >= number_of_units * 18) & (units < number_of_units * 19):
        res = dist + (diff*18)
    elif (units >= number_of_units * 19) & (units < number_of_units * 20):
        res = dist + (diff*19)
    elif (units >= number_of_units * 20) & (units < number_of_units * 21):
        res = dist + (diff*20)
    elif (units >= number_of_units * 21) & (units < number_of_units * 22):
        res = dist + (diff*21)
    elif (units >= number_of_units * 22) & (units < number_of_units * 23):
        res = dist + (diff*22)
    elif (units >= number_of_units * 23) & (units < number_of_units * 24):
        res = dist + (diff*23)
    elif (units >= number_of_units * 24) & (units < number_of_units * 25):
        res = dist + (diff*24)
    elif (units >= number_of_units * 25) & (units < number_of_units * 26):
        res = dist + (diff*25)
    elif (units >= number_of_units * 26) & (units < number_of_units * 27):
        res = dist + (diff*26)
    elif (units >= number_of_units * 27) & (units < number_of_units * 28):
        res = dist + (diff*27)
    elif (units >= number_of_units * 28) & (units < number_of_units * 29):
        res = dist + (diff*28)
    elif (units >= number_of_units * 29) & (units < number_of_units * 30):
        res = dist + (diff*29)
    elif (units >= number_of_units * 30):
        res = dist + (diff*30)
    else :
        res = dist 
    
        
    return res

In [ ]:
## 세대수별 차등 반경으로 집계시 사용
# distance_list = [6000,7000, 8000, 9000]
distance_list = [distinace]

for j, i in enumerate(distance_list):
    # 실행중
    dt = datetime.datetime.now()
    fl_dt = str(dt.year) + str(dt.month) + str(dt.day)
    
    ## count 대상정의
    query = "검색대상"
    distance = i
    col_nm = str(fl_dt) + "_" + str(query) + "_" + str(distance)

    ## 반경 세대별 수정
    m_dt1["distance_mdf"] = m_dt1.apply(lambda x: cal_distance_unit(x["number_of_units"], distance), axis = 1)
    
    ## count 수행
    m_dt1[col_nm] = m_dt1.apply(lambda x: get_count_from_KAKAO(query, x['x'], x['y'], x["distance_mdf"]), axis = 1)

    m_dt2 = m_dt1.rename(columns={col_nm : 'Count'})
    dt = datetime.datetime.now()
    fl_dt = str(dt.year) + str(dt.month) + str(dt.day)
    
    m_dt2["year"] = str(dt.year)
    m_dt2["month"] = str(dt.month) 
    m_dt2["date"] = str(dt.day)
    m_dt2["cnt_nm"] = 0
    
    
    ## cnt_nm : 검색어 + n번째 반경 조정 + 세대수별 반경 간격 + 조정된 반경
    for n in m_dt2.index:
        m_dt2["cnt_nm"][n] = query + "_" + str(j + 1) + "_" + str(diff_a) + "_"+ str(m_dt2["distance_mdf"][n])
    
    m_dt2["distance"] = distance
    m_dt2['schedul_yn'] = 'Y'
    m_dt2["created_at"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


    m_dt2 = m_dt2[['kbm_apartment_id','year',
           'month', 'date', 'cnt_nm', 'distance','Count','schedul_yn', 'created_at']]
    rsult_lenth = len(m_dt2)

    db_con = pymysql.connect(
        user='', 
        passwd='', 
        host='', 
        db='', 
        charset=''
    )

    # Insert 수행 전 건수 확인
    cursor = db_con.cursor(pymysql.cursors.DictCursor)


    sql2 = """insert into () VALUES ())
    """

    cursor.executemany(sql2, m_dt2.values.tolist())
    db_con.commit()

    del m_dt2
    db_con.close()

#     rsult_lenth2 = (aft_lenth - pre_lenth)
    print("Insert result : " , query , i )